In [ ]:
import socket
import logging
import time

# Configure logging
logging.basicConfig(filename='modbuslog.log', filemode='w', level=logging.DEBUG,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Define the file path
FILE_PATH = 'mutated_packets(epoch20_20k_1).txt'

# TCP connection details
HOST = 'localhost'
PORT = 502

# Function to read packets from file
def read_packets_from_file(file_path):
    packets = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the line into a list of integers (hex to int)
            packet = bytes([int(x, 16) for x in line.split()])
            packets.append(packet)
    return packets

# Function to send a packet and receive a response
def send_packet(packet):
    try:
        # Create socket and connect to server
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)  # Set a timeout for the socket
        sock.connect((HOST, PORT))

        # Send data
        start_time = time.time()
        logging.debug(f"SEND: {format_bytes_as_hex(packet)}")
        sock.sendall(packet)
        print(f"Packet: {format_bytes_as_hex(packet)}")


        # Receive response
        response = sock.recv(1024)
        print(f"Response: {format_bytes_as_hex(response)}")

        logging.debug(f"RECV: {format_bytes_as_hex(response)}")
        end_time = time.time()
        sock.close()

        # Check for overtime return
        if (end_time - start_time) > 10:  # If it took longer than 10 seconds to get a response
            logging.error(f"Overtime return for packet: {packet}")
            return False, "overtime_return"
        if len(response) < 8:
            logger.debug("Response too short")
            return "Exception code: Response too short", response
        
        # Check the eighth byte for the exception code
        function_code = packet[7]  # The original function code sent
        exception_code = function_code + 0x80
        
        if response[7] == exception_code:
            logging.error(f"Exception message received for packet: {packet}")
            logger.debug(f"Exception code: {exception_code}")
            return False, response

        else:
            return True, None   # Normal response
    except socket.timeout:
        logging.error(f"Socket timeout for packet: {packet}")
        return False, "timeout"
    except socket.error as e:
        logging.error(f"Socket error: {e} for packet: {packet}")
        return False, "socket_error"
    except Exception as e:
        logging.error(f"Exception: {e} for packet: {packet}")
        return False, "general_exception"

def format_bytes_as_hex(byte_sequence):
    return ' '.join([f'0x{byte:02x}' for byte in byte_sequence])

def detect_crash():
    try:
        # Attempt to connect to the server to check if it's up
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((HOST, PORT))
        sock.close()
        return False  # No crash detected
    except Exception as e:
        logging.error(f"Crash detected: {e}")
        return True   # Crash detected

# Read packets from file
packets = read_packets_from_file(FILE_PATH)

# Total number of packets
total_packets = len(packets)

# Count recognized normal packets (N_legal) and abnormal packets (N_unusual)
normal_responses = 0
abnormal_responses = 0
crashes_detected = {
    "timeout": 0,
    "socket_error": 0,
    "overtime_return": 0,
    "exception_message": 0,
    "general_exception": 0,
    "server_crash": 0
}

for packet in packets:
    success, error_type = send_packet(packet)
    if success:
        normal_responses += 1
    else:
        abnormal_responses += 1
        if error_type:
            crashes_detected[error_type] += 1
        if detect_crash():
            crashes_detected["server_crash"] += 1

# Calculate TSRR (Test System Reception Rate)
tsrr = normal_responses / total_packets if total_packets > 0 else 0

# Calculate TSAR (Test System Abnormal Rate)
total_crashes = sum(crashes_detected.values())
tsar = total_crashes/ total_packets if total_packets > 0 else 0

print(f"Test System Reception Rate (TSRR): {tsrr * 100:.2f}%")
print(f"Test System Abnormal Rate (TSAR): {tsar * 100:.2f}%")
print(f"Number of crashes detected: {total_crashes}")
print(f"Details of crashes detected: {crashes_detected}")

